<a href="https://colab.research.google.com/github/tguerand/SentimentAnalysis/blob/master/bin/data_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Access Path

In [ ]:
cd /content/drive/My\ Drive/Colab\ Projects/SentimentAnalysis/data/

/content/drive/My Drive/Colab Projects/SentimentAnalysis/data


# Import Libraries

In [ ]:
import numpy as np
import pandas as pd



#Import Data

In [ ]:
df=pd.read_csv("training.csv",encoding='latin-1')
df.columns=['target', 't_id', 'created_at', 'query', 'user', 'text']
df.head()

In [ ]:
print(df["text"].head(100).to_string())

0     is upset that he can't update his Facebook by ...
1     @Kenichan I dived many times for the ball. Man...
2       my whole body feels itchy and like its on fire 
3     @nationwideclass no, it's not behaving at all....
4                         @Kwesidei not the whole crew 
5                                           Need a hug 
6     @LOLTrish hey  long time no see! Yes.. Rains a...
7                  @Tatiana_K nope they didn't have it 
8                             @twittera que me muera ? 
9           spring break in plain city... it's snowing 
10                           I just re-pierced my ears 
11    @caregiving I couldn't bear to watch it.  And ...
12    @octolinz16 It it counts, idk why I did either...
13    @smarrison i would've been the first, but i di...
14    @iamjazzyfizzle I wish I got to watch it with ...
15    Hollis' death scene will hurt me severely to w...
16                                 about to file taxes 
17    @LettyA ahh ive always wanted to see rent 